In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
nd = 128  # 計算領域の１辺の分割数

In [3]:
RR = 8.3145

In [4]:
def step(s1, s2):
    # 時間変化量の計算
    s1ddtt, s2ddtt = dsdt(s1, s2)

    # phase fieldの時間発展の計算
    s1 = s1 + delt * (s1ddtt + ds_fac * (2.0 * np.random.rand(nd, nd) - 1.0))
    s2 = s2 + delt * (s2ddtt + ds_fac * (2.0 * np.random.rand(nd, nd) - 1.0))

    # sの変域(0<=s<=1)の補正
    s1, s2 = comp(s1, s2)
    return s1, s2

In [5]:
def dsdt(s1, s2):
    # 勾配ポテンシャル
    s1k_su = -kappa_s1 * laplacian(s1)
    s2k_su = -kappa_s2 * laplacian(s2)

    # 態歪場のフ−リエ変換 ep11
    ep110 = eta_s1[1, 1] * s1 ** 2 + eta_s2[1, 1] * s2 ** 2
    ep11q0 = np.fft.fft2(ep110)

    # 変態歪場のフ−リエ変換 ep22
    ep220 = eta_s1[2, 2] * s1 ** 2 + eta_s2[2, 2] * s2 ** 2
    ep22q0 = np.fft.fft2(ep220)

    # 変態歪場の平均値の算出
    ep11_0 = np.average(ep110)
    ep22_0 = np.average(ep220)

    # 拘束歪変動量ec11の計算
    ec11 = np.fft.ifft2(Z11ep * ep11q0 + Z12ep * ep22q0).real

    # 拘束歪変動量ec22の計算
    ec22 = np.fft.ifft2(Z21ep * ep11q0 + Z22ep * ep22q0).real

    # ポテンシャルと発展方程式の計算

    # 化学ポテンシャルの計算
    s12 = s1 ** 2  # 演算回数の削減のため、先に計算
    s13 = s1 * s12
    s22 = s2 ** 2
    s23 = s2 * s22
    s1k_chem = AA0 * AA1 * s1 + AA0 * AA2 * s13 + AA0 * AA3 * s12 * s13 + AA0 * AA4 * s1 * s22 + AA0 * AA5 * s1 * s22 * (
            2.0 * s12 + s22)
    s2k_chem = AA0 * AA1 * s2 + AA0 * AA2 * s23 + AA0 * AA3 * s22 * s23 + AA0 * AA4 * s2 * s12 + AA0 * AA5 * s2 * s12 * (
            2.0 * s22 + s12)

    # 弾性ポテンシャルの計算
    ep11T = ep110 - ep11_0 - ec11 - ep11_a
    ep22T = ep220 - ep22_0 - ec22 - ep22_a
    s1k_str = 2.0 * s1 * (ep11T * ((lam0 + 2.0 * mu0) * eta_s1[1, 1] + lam0 * eta_s1[2, 2]) + ep22T * (
            (lam0 + 2.0 * mu0) * eta_s1[2, 2] + lam0 * eta_s1[1, 1]))
    s2k_str = 2.0 * s2 * (ep11T * ((lam0 + 2.0 * mu0) * eta_s2[1, 1] + lam0 * eta_s2[2, 2]) + ep22T * (
            (lam0 + 2.0 * mu0) * eta_s2[2, 2] + lam0 * eta_s2[1, 1]))

    s1ddtt = -smob * (s1k_chem + s1k_su + s1k_str)
    s2ddtt = -smob * (s2k_chem + s2k_su + s2k_str)

    return s1ddtt, s2ddtt

In [6]:
# 二次元のラプラシアン
def laplacian(array):
    result = -4 * array  # 元の配列の-4倍の配列を用意
    result += np.roll(array, 1, 0)  # それぞれの軸に対して-1, 1要素ずつずらした配列を足し込む(境界は周期境界)
    result += np.roll(array, -1, 0)
    result += np.roll(array, 1, 1)
    result += np.roll(array, -1, 1)
    return result

In [7]:
def comp(s1, s2):
    s1 = np.clip(s1, -1., 1.)
    s2 = np.clip(s2, -1., 1.)
    return s1, s2

In [8]:
def init():
    fac1 = 0.9
    s1 = fac1 * (2. * np.random.rand(nd, nd) - 1.)
    s2 = fac1 * (2. * np.random.rand(nd, nd) - 1.)
    return s1, s2

In [9]:
delt = 0.05  # 時間きざみ入力

temp = 873.0  # 温度[K]
al = 200.0 * 1.0e-09  # 計算領域[m]
b1 = al / nd  # 差分ブロックの長さ[m]

time1 = 0
time1max = 10000

smob = 1.0  # マルテンサイト変態ダイナミクスの緩和係数
ds_fac = 0.1  # pase fieldの揺らぎ係数

AA0 = 3.82027e+03  # マルテンサイト変態の化学的駆動力[J/mol]
AA0 = AA0 / RR / temp  # 無次元化
AA1 = 0.1  # 化学的駆動力定数
AA2 = -4.0 * AA1 - 12.0
AA3 = 3.0 * AA1 + 12.0
AA4 = AA5 = AA6 = 4.0

kappa_s1 = 1.5e-14  # 勾配エネルギ−定数[Jm^2/mol]
kappa_s1 = kappa_s1 / RR / temp / b1 ** 2  # 無次元化
kappa_s2 = kappa_s1

a1_c = b1_c = c1_c = 3.6468e-10  # Fe(fcc)の格子定数[nm]
atom_n = 4.0
vm0 = 6.02E23 * a1_c * b1_c * c1_c / atom_n  # モル体積の計算（fccを仮定）[m^3/mol]

In [10]:
# s1場の変態歪の設定
eta_s1 = np.zeros((4, 4))
eta_s1[1, 1] = -0.06
eta_s1[2, 2] = 0.06
eta_s1[3, 3] = 0.0

In [11]:
# s2場の変態歪の設定
eta_s2 = np.zeros((4, 4))
eta_s2[1, 1] = eta_s1[2, 2]
eta_s2[2, 2] = eta_s1[1, 1]
eta_s2[3, 3] = 0.0

In [12]:
# 弾性定数(Fe(fcc)の場合)
el_fac = 1.0e+11 * vm0 / RR / temp
c11 = 1.54 * el_fac
c44 = 0.77 * el_fac
c12 = 1.22 * el_fac
# c12=c11-2.0*c44
lam0 = c12
mu0 = c44  # ラーメの定数
nu0 = lam0 / 2.0 / (lam0 + mu0)  # ポアソン比

In [13]:
# 外力の設定
sig22_a = 0.0  # ここでは外力を0に設定
# sig22_a = 0.1
ep11_a = -lam0 / 4.0 / mu0 / (lam0 + mu0) * sig22_a  # 平面歪を想定
ep22_a = (lam0 + 2.0 * mu0) / 4.0 / mu0 / (lam0 + mu0) * sig22_a
ep12_a = ep21_a = 0.0

In [14]:
# 逆空間の演算で使用する数値の準備
jj, ii = np.meshgrid(np.fft.fftfreq(nd), np.fft.fftfreq(nd))  # ii, jj の二次元配列を準備(順番に注意)
alnn = (ii ** 2 + jj ** 2) ** 0.5
alnn[alnn == 0] = 1.0
nxx = (ii / alnn) ** 2
nyy = (jj / alnn) ** 2
Z11ep = nxx * (2.0 * (1.0 - nu0) - nxx - nu0 / (1.0 - nu0) * nyy) / (1.0 - 2.0 * nu0)  # 計算ループ中で不変の部分を計算しておく
Z12ep = nxx * (2.0 * nu0 - nyy - nu0 / (1.0 - nu0) * nxx) / (1.0 - 2.0 * nu0)
Z21ep = nyy * (2.0 * nu0 - nxx - nu0 / (1.0 - nu0) * nyy) / (1.0 - 2.0 * nu0)
Z22ep = nyy * (2.0 * (1.0 - nu0) - nyy - nu0 / (1.0 - nu0) * nxx) / (1.0 - 2.0 * nu0)

In [15]:
# 初期場の生成
s1, s2 = init()

In [ ]:
for time1 in range(time1max):
    s1, s2 = step(s1, s2)  # 濃度場の更新

    if time1 % 50 == 0:  # 50タイムステップごとに描画
        # 色の管理用のflagを計算
        flag0 = s1 >= 0.
        flag1 = s2 >= 0.

        # 色の計算
        c1r = np.where(flag0, s1, 0.)
        c1g = 0.
        c1b = np.where(flag0, 0., -s1)
        c2r = np.where(flag1, 0., -s2)
        c2g = np.abs(s2)
        c2b = 0.

        # 描画用の配列を生成
        draw = np.dstack((c1r + c2r, c1g + c2g, c1b + c2b))
        draw = np.clip(draw, 0, 1)

        plt.clf()  # 描画内容クリア
        plt.imshow(draw)  # 濃度場の描画
        plt.pause(0.1)  # 0.1秒間表示